In [92]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [93]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [94]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [128]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) 

data_dev = data[0:7000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[7000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

data_dev = test_data.T
Y_test = data_dev[0]
X_test = data_dev[1:n]
X_test = X_test / 255.



In [97]:
def init_params():
    W1 = np.random.rand(25, 784) - 0.5
    b1 = np.random.rand(25, 1) - 0.5
    W2 = np.random.rand(25, 25) - 0.5
    b2 = np.random.rand(25, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [99]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [126]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.05, 10000)

Iteration:  0
[10 10 10 ... 10 10 10] [21 17 23 ...  8 19 12]
0.03534588120263994
Iteration:  10
[15  5 18 ... 18 18  3] [21 17 23 ...  8 19 12]
0.05402102175507211
Iteration:  20
[24 19 19 ... 18 18  3] [21 17 23 ...  8 19 12]
0.05020777316059643
Iteration:  30
[24 19 19 ... 13 18  3] [21 17 23 ...  8 19 12]
0.05304326570520655
Iteration:  40
[24 19 19 ... 13 18  3] [21 17 23 ...  8 19 12]
0.055585431434857
Iteration:  50
[24 19 19 ... 13 14  3] [21 17 23 ...  8 19 12]
0.05680762649718895
Iteration:  60
[24 19 19 ... 13 14  3] [21 17 23 ...  8 19 12]
0.059056465411879734
Iteration:  70
[24 19  4 ... 13 14  3] [21 17 23 ...  8 19 12]
0.06458078709362014
Iteration:  80
[24 19  4 ... 13 14  3] [21 17 23 ...  8 19 12]
0.06829626008310927
Iteration:  90
[24 19  4 ... 13 14  3] [21 17 23 ...  8 19 12]
0.07215839648007823
Iteration:  100
[24 19  4 ... 13 14  3] [21 17 23 ...  8 19 12]
0.07753605475433879
Iteration:  110
[24 19  4 ... 13 14  3] [21 17 23 ...  8 19 12]
0.08183818137374725
Iter

In [130]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [129]:
# test_prediction(0, W1, b1, W2, b2)
# test_prediction(1, W1, b1, W2, b2)
# test_prediction(2, W1, b1, W2, b2)
# test_prediction(3, W1, b1, W2, b2)
pred_data_r= make_predictions(X_test,W1, b1, W2, b2)
pred_data= make_predictions(X_dev,W1, b1, W2, b2)

print(get_accuracy(pred_data,Y_dev))
print(pred_data_r)


[24 19  5 ...  4  6 18] [24 19  5 ...  4  6 18]
0.9252857142857143
[ 6  5 10 ...  2  4  2]


In [131]:
submit = pd.read_csv('sample_submission.csv')
submit.label=(pred_data_r)
submit.to_csv('predictions.csv', index=False)